In [6]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re
from urllib.parse import unquote
import os
import time
from bs4 import BeautifulSoup
import random
import numpy as np
from datetime import datetime
import warnings
from urllib.parse import urlparse
from urllib.parse import urljoin
import urllib3

## SMM Scrapper


### get_accounts_websites_for_smm_scrap()
This scrapper takes the homepage of a website and searches for its smm links in the href values of the page

Below, we use pandas to read in all websites and IDs for our accounts, which then use numpy to turn these into a tuple

In [7]:
def get_accounts_websites_for_smm_scrap():  
    global tuple_list
    csv_of_urls = fpd.read_csv('C:/Users/Nick/Desktop/scrap-vet-accounts/Important Files/All 27k Websites Homepage Scrapped 10_13_23.csv')
    web_and_ID = csv_of_urls[['ID','Website']]
    tuple_list = [tuple(x) for x in web_and_ID.to_numpy()]

    return tuple_list

### scrap_smm_info_from_pages()

The function below starts by breaking the tuple apart, we then create a list of the social media patterns to search for
Then we make get requests to each site, and use list comp to search for each href value on the page that contains the smm pattern

This all gets added to a set to get rid of duplicates, and that set is then turned into a list, that is then turned into a dictonary, where the key is the ID, and the value is the smm link

smm_dict then holds the values for each website

In [8]:
def scrap_smm_info_from_pages(tuple_list):

    global smm_dict

    smm_dict = {}
    warnings.filterwarnings('ignore', category=requests.packages.urllib3.exceptions.InsecureRequestWarning)
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36'
        }
    
    social_media_patterns = ['www.facebook.com', 'www.instagram.com', 'www.linkedin.com', 'www.twitter.com','facebook.com','instagram.com','linkedin.com','twitter.com', 'x.com']

    for id4, smm_find in tuple_list:
        smm_links = set()  
        try:
            response2 = requests.get(smm_find, timeout=8, headers=headers, verify=False)
            soupy = BeautifulSoup(response2.text, 'html.parser')
            body = soupy.find('body')

            if body is not None:
                href_values = [link.get('href') for link in body.find_all('a') if link.get('href')]

                for href in href_values:
                    for pattern in social_media_patterns:
                        if pattern in href:
                            smm_links.add(href) 

        except requests.exceptions.RequestException as e:
            pass

        if smm_links:
            smm_dict[id4] = list(smm_links)  
        time.sleep(2)

    return smm_dict

### create_df_cleaned_from_smm ()

Below, we then create a dataframe from the dictonary, were we then apply list comp to see if each value contains one of the platforms
we then break those different links into their own columns, we also drop any duplicates again and return a dataframe

In [9]:
def create_df_cleaned_from_smm(smm_dict):  
    global df_of_smm_opt

    df_of_smm_opt = pd.DataFrame(smm_dict.items(), columns=['ID', 'Social Media Links'])
    
    platforms = ['Facebook', 'Instagram', 'LinkedIn','Twitter']

    for platform in platforms:
        df_of_smm_opt[platform] = df_of_smm_opt['Social Media Links'].apply(
            lambda links: [link for link in links if platform.lower() in link.lower()][0] if any(platform.lower() in link.lower() for link in links) else None
        )
    df_of_smm_opt = df_of_smm_opt.drop('Social Media Links', axis=1)
    
    return df_of_smm_opt

### if __name__ == '__main__':

We then use all the functions above and their returned objects to run the entire script

This should return all the smm links found on the homepage, and output the dataframe in the global df_of_smm_opt variable which contains the dataframe that can be downloaded in a needed format

70% Success Rate

In [ ]:
if __name__ == '__main__':
    create_df_cleaned_from_smm(scrap_smm_info_from_pages(get_accounts_websites_for_smm_scrap()))

df_of_smm_opt

save = df_of_smm_opt.to_csv('smm_25k_homepage_scrapped.csv')

In [13]:
df_of_smm_opt

,ID,Facebook,Instagram,LinkedIn,Twitter
0,0016g00001yUhIzAAK,https://www.facebook.com/eastportanimalhospital/,https://www.instagram.com/eastportanimalhospital/,None,None
1,0016g00002Sy5yUAAR,http://www.facebook.com/ponderosavet/,https://www.instagram.com/ponderosaveterinary/...,None,None
2,0016g00002Sy5yVAAR,https://www.facebook.com/CountrysideKalispellVet/,None,None,None
3,0016g00002Sy5ydAAB,https://www.facebook.com/CalmAnimalCare,None,None,None
4,0016g00002Sy5yJAAR,https://www.facebook.com/burntforkvetclinic/,None,None,https://twitter.com/vet_matrix
...,...,...,...,...,...
17078,0016g00001wo5WXAAY,https://www.facebook.com/Beattie-Pet-Hospital-...,https://www.instagram.com/beattiepethospitalea...,None,None
17079,0016g000017JaUrAAK,https://www.facebook.com/MyrtleGroveAnimalHosp...,https://www.instagram.com/myrtlegroveanimalhos...,None,None
17080,0016g00001vc8yeAAA,https://www.facebook.com/profile.php?id=100063...,None,None,https://twitter.com/RedlandsPetCl
17081,0016g00002NrMJVAA3,https://www.facebook.com/glovecitiesvethospital,None,None,None
